Version 1.0.1

# Pandas basics 

Hi! In this programming assignment you need to refresh your `pandas` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`]()`s to solve the task. 

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

from grader import Grader

In [ ]:
DATA_FOLDER = '../readonly/final_project_data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

The dataset we are going to use is taken from the competition, that serves as the final project for this course. You can find complete data description at the [competition web page](https://www.kaggle.com/c/competitive-data-science-final-project/data). To join the competition use [this link](https://www.kaggle.com/t/1ea93815dca248e99221df42ebde3540).

## Grading

We will create a grader instace below and use it to collect your answers. When function `submit_tag` is called, grader will store your answer *locally*. The answers will *not* be submited to the platform immediately so you can call `submit_tag` function as many times as you need. 

When you are ready to push your answers to the platform you should fill your credentials and run `submit` function in the <a href="#Authorization-&-Submission">last paragraph</a>  of the assignment.

In [ ]:
grader = Grader()

# Task

Let's start with a simple task. 

<ol start="0">
  <li><b>Print the shape of the loaded dataframes and use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows. Examine the features you are given.</b></li>
</ol>

In [ ]:
print(transactions.shape)
print(transactions.head())
print(items.shape)
print(items.head())
print(item_categories.shape)
print(item_categories.head())
print(shops.shape)
print(shops.head())

Now use your `pandas` skills to get answers for the following questions. 
The first question is:

1. ** What was the maximum total revenue among all the shops in September, 2014?** 


* Hereinafter *revenue* refers to total sales minus value of goods returned.

*Hints:*

* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 9` in order to select target subset of dates.
* You may work with `date` feature as with srings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument.

In [ ]:
print(items.head(-5))

In [ ]:
unique_shops = list(set(shops.shop_id))
revenue = {key: 0 for key in unique_shops}

for i in range(transactions.shape[0]):
    print(i if i%100 else '\r', end='\t')
    if (transactions.date[i][3:5] == '09' and transactions.date[i][6:10] == '2014'):
        revenue[transactions.shop_id[i]] += (transactions.item_price[i])*(transactions.item_cnt_day[i])


max_revenue = max(revenue.values())

grader.submit_tag('max_revenue', max_revenue)

Great! Let's move on and answer another question:

<ol start="2">
  <li><b>What item category generated the highest revenue in summer 2014?</b></li>
</ol>

* Submit `id` of the category found.
    
* Here we call "summer" the period from June to August.

*Hints:*

* Note, that for an object `x` of type `pd.Series`: `x.argmax()` returns **index** of the maximum element. `pd.Series` can have non-trivial index (not `[1, 2, 3, ... ]`).

In [ ]:
category_groups = items.groupby(by=['item_category_id'])
print(category_groups.groups[0])

item_groups = transactions.groupby(by=['item_id'], axis=0)
print(item_groups.groups.keys())

In [ ]:
max_revenue = 0
max_id = 0

for category_id in category_groups.groups.keys():
    category_revenue = 0
    print("Checking category %d"%(category_id), end=' ')
    
    for item_id in category_groups.groups[category_id]:
#         print("Item %d"%(item_id), end=' ')
        if (item_id in item_groups.groups.keys()):
            for i in item_groups.groups[item_id]:
#                 print("%d" % i, end=' ')

                if ((6 <= int(transactions.date[i][3:5]) <= 8) and transactions.date[i][6:10] == '2014'):
                    category_revenue += transactions.item_price[i] * transactions.item_cnt_day[i]
                
    if(category_revenue > max_revenue):
        max_revenue = category_revenue
        max_id = category_id
        print('\rHighest revenue so far: Category %d\n' % max_id, end='')
    print('\r', end=' ')
category_id_with_max_revenue = max_id
grader.submit_tag('category_id_with_max_revenue', category_id_with_max_revenue)

<ol start="3">
  <li><b>How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?</b></li>
</ol>

* Let's assume, that the items are returned for the same price as they had been sold.

In [ ]:
# YOUR CODE GOES HERE
price_changes = {item_id:len(set(transactions.item_price.loc[item_groups.groups[item_id]])) for item_id in item_groups.groups.keys()}

num_items_constant_price = list(price_changes.values()).count(1)
grader.submit_tag('num_items_constant_price', num_items_constant_price)

Remember, the data can sometimes be noisy.

<ol start="4">
  <li><b>What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014?</b></li>
</ol>

* Fill `total_num_items_sold` and `days` arrays, and plot the sequence with the code below.
* Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see [link](https://math.stackexchange.com/questions/496627/the-difference-between-unbiased-biased-estimator-variance)). Compute ***unbiased*** estimate (use the right value for `ddof` argument in `pd.var` or `np.var`).

In [ ]:
shop_id = 25

data = transactions[transactions['shop_id'] == shop_id]
data = data[data['date'].str.endswith('12.2014')]
data = data['item_cnt_day'].groupby(by=data['date']).sum()

total_num_items_sold = data.values
days = data.index.to_datetime()

# Plot it
plt.plot(days, total_num_items_sold)
plt.ylabel('Num items')
plt.xlabel('Day')
plt.title("Daily revenue for shop_id = 25")
plt.show()

total_num_items_sold_var = data.var()
grader.submit_tag('total_num_items_sold_var', total_num_items_sold_var)

## Authorization & Submission
To submit assignment to Cousera platform, please, enter your e-mail and token into the variables below. You can generate token on the programming assignment page. *Note:* Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = ""
STUDENT_TOKEN = ""
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Well done! :)